 After learning to map our class and create a session, we can move on to the next step, querying and filtering data from a table. To achieve this, we will use ```query()``` and ```filter()``` methods, but first, we will discuss the Query constructor object and how to use it.

# Query constructor object

- According to the documentation, a Query construction object is the source of all SELECT statements generated by the ORM. What does it mean exactly? A Query object will generate a SELECT statement for columns of our table and rename each column to a variable name as per PEP convention. Here's what it may look like. Once passed, it will create the animals_petname variable for the table called Animals with the petname column. After this, you'll be able to access the values in the corresponding column via this variable.


- To use a Query object, you first need to import it; it will also require a mapped class. Let's have a look at the snippet below where we define the mapped class from the previous topic:


In [ ]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Query  # new import


Base = declarative_base()


class Animals(Base):
    __tablename__ = 'animals'

    id = Column(Integer, primary_key=True)
    petname = Column(String(30))
    age = Column(Integer)
    weight = Column(Integer)

Now, we can pass our mapped class to the Query constructor as an argument, and it will return a query object:

In [ ]:
query = Query(Animals)  # <class 'sqlalchemy.orm.query.Query'>


The query variable is an instance of the Query constructor. It allows us to use all Query methods. We will discuss it later in this topic.

- Let's take a closer look at our query object.



In [ ]:
print(query)
# SELECT animals.id AS animals_id, 
# animals.petname AS animals_petname, 
# animals.age AS animals_age, 
# animals.weight AS animals_weight 
# FROM animals


It is magic in action — the Query constructor generates an SQL SELECT statement for each column of the animals table and renames each column to a variable following the PEP convention. 

# Selecting from the table
 
- We have learned to create an SQL statement, but before we start working with the data in the table, we also need to create Session(). You already know how to do it:

In [ ]:
from sqlalchemy.orm import Query, sessionmaker
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:", echo=True)
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

Now, let's consider two ways of how we can handle data. The first one is the Query constructor, but this time we should also specify our session. It can be passed either as a second argument to Query() or you can use the with_session() method; both options are identical:

In [ ]:
query = Query(Animals, session)

# The code above is equivalent to:
query = Query(Animals)
query = query.with_session(session)

The second way is using the query() method of our session:



In [ ]:
query = session.query(Animals)


# Retrieving all values

Now, once we have the query object, we can start working with the values from our table. First, let's add some values:

In [ ]:
session.add(Animals(petname="Billy", age=1, weight=8))
session.add(Animals(petname="Susan", age=4, weight=12))

session.commit()

We will start with the all() method.



In [ ]:
all_rows = query.all()


It returns a list of tuples with the values from our query. We can use a for loop to process them. Each object inside the all_rows variable is an instance of the Animals class:

In [ ]:
for row in all_rows:
    print(type(row))  # <class '__main__.Animals'>

Now, we can retrieve the values by the attribute names that are defined inside the Animals class:

In [ ]:
for row in all_rows:
    print(f"Pet name: {row.petname}, age: {row.age}, weight: {row.weight}")
# Pet name: Billy, age: 1, weight: 8
# Pet name: Susan, age: 4, weight: 12

# Retrieving certain values

What if you don't want to select all attributes, as you're looking only for certain ones? You can specify the attributes you want by passing the class and the name of the attribute as a parameter to your query object:

In [ ]:
query = session.query(Animals.petname)
print(query)
# SELECT animals.petname AS animals_petname FROM animals

You can pass as many attributes as you want but separate them with a comma. Since we are selecting attributes directly, the output will contain a tuple with the attributes of the Animals class:

In [ ]:
query = session.query(Animals.petname, Animals.age)
for petname, age in query:
    print(petname, age)
# Billy 1
# Susan 4

Another useful method is count(). It will return the number of rows that match a special criterion. Remember that the number of rows is not the same as the number of values. The number of values and rows will be the same only when you select one field from the table. For example, we have 6 values and 2 rows in the snippet above, so the method returns 2.

In [ ]:
query = session.query(Animals)
print(query.count()) # 2

# Filtering the table
To filter the data, you can use the filter() method. No surprise that you need to pass the exact parameters of what you want to filter as a parameter. For example, if you want to filter all animals with the Billy pet name, use this: ```Animals.petname == "Billy"```. It is a boolean expression by nature:

In [ ]:
query = session.query(Animals)
for row in query.filter(Animals.petname == "Billy"):
    print(row.petname, row.age, row.weight) # Billy 1 8

The above code will generate the following SQL statement:



In [ ]:
# SELECT animals.id AS animals_id, animals.petname AS animals_petname, 
# animals.age AS animals_age, animals.weight AS animals_weight 
# FROM animals
# WHERE animals.petname = ?

In WHERE animals.petname = ? the question mark will be replaced by the value we are looking for, in this case, Billy.

There are no limits on what you can filter. For example, you can use more than one parameter and only a few specific attributes. You can pass any number of attributes and any number of parameters. Also, since a parameter is a boolean expression, you can use another boolean operator as well, for example >=. Let's see an example in the code snippet below:

In [ ]:
query = session.query(Animals.age, Animals.weight)

age_gr_than = Animals.age > 2
weight_eq_gr_than = Animals.weight >= 8

for age, weight in query.filter(age_gr_than, weight_eq_gr_than):
    print(f"Pet age: {age}, Pet weight: {weight}")  # Pet age: 4, Pet weight: 12

First of all, we specify the attributes we would like to select — Animals.age and Animals.weight. Then we create two boolean expressions: the age and weight of an animal. We have only one animal that satisfies both conditions, so it produces only one result. If the criterion does not satisfy any field, the query will return nothing.